In [1]:
%env OGDF_BUILD_DIR=~/OGDF-master/build-debug
from ogdf_python import ogdf, cppinclude
cppinclude("ogdf/basic/graph_generators/randomized.h")
cppinclude("ogdf/layered/SugiyamaLayout.h")
cppinclude("ogdf/basic/graph_generators/deterministic.h")
cppinclude("ogdf/energybased/FMMMLayout.h")


G = ogdf.Graph()
ogdf.setSeed(1)
ogdf.gridGraph(G, 3, 3, True,True)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
GA.destroyAttributes(ogdf.GraphAttributes.nodeId)

for n in G.nodes:
    GA.label[n] = "N%s" % n.index()
    GA.x[n] = (n.index() %3) * 50 
    GA.y[n] = (n.index()//3) * 50
    
#or adj in G.nodes[0].adjEntries:
#   print(adj.twinNode().index())
#   if adj.twinNode().index() == 9:
#       e = adj.theEdge()
#       GA.bends[e].pushBack(ogdf.DPoint(-100,-100)) 
    
#SL = ogdf.SugiyamaLayout()
#SL=ogdf.FMMMLayout();
 
#SL.useHighLevelOptions(True);
#SL.unitEdgeLength(15.0);
#SL.newInitialPlacement(True);
#SL.qualityVersusSpeed(ogdf.FMMMOptions.QualityVsSpeed.GorgeousAndEfficient);
#SL.call(GA)
ogdf.GraphIO.write(GA, "sugiyama-simple.graphml")

GA

env: OGDF_BUILD_DIR=~/OGDF-master/build-debug


In [1]:
%env OGDF_BUILD_DIR=~/OGDF-master/build-debug
from ogdf_python import ogdf, cppinclude
cppinclude("ogdf/basic/graph_generators/deterministic.h")

# 0 -- > x
# |
# v
#  y

def add_bends(line, *points):
    for point in points:
        line.pushBack(ogdf.DPoint(*point))

def find_wrap(GA, e):
    s,t = e.source(), e.target()
    wrap_h = GA.x[t] < GA.x[s]
    wrap_v = GA.y[t] < GA.y[s]
    if wrap_h and wrap_v:
        delta_x = abs(GA.x[t] - GA.x[s])
        delta_y = abs(GA.y[t] - GA.y[s])
        if delta_x > delta_y:
            wrap = "hv"
        else:
            wrap = "vh"
    elif wrap_h:
        wrap = "h"
    elif wrap_v:
        wrap = "v"
    else:
        wrap = ""
    return wrap

def apply_wrap(GA, box, e, wrap, margin=50):
    if not wrap:
        return
    
    s,t = e.source(), e.target()
    left = box.p1().m_x
    top = box.p1().m_y
    right = box.p2().m_x
    bottom = box.p2().m_y
    
    GA.bends[e].clear()
    if wrap[0] == "h":
        add_bends(GA.bends[e],
            (right + margin, GA.y[s]),
            (right + margin, top - margin),
            (left - margin, top - margin),
            (left - margin, GA.y[t]),
        )
    elif wrap[0] == "v":
        add_bends(GA.bends[e],
            (GA.x[s], bottom + margin),
            (left - margin, bottom + margin),
            (left - margin, top - margin),
            (GA.x[t], top - margin),
        )


def layout_edges(GA, box):
    wraps = ogdf.EdgeArray[str](GA.constGraph(), "")
    wraps_h = []
    wraps_v = []
    for e in GA.constGraph().edges:
        w = wraps[e] = find_wrap(GA, e)
        if w and w[0] == "h":
            wraps_h.append(e)
        elif w and w[0] == "v":
            wraps_v.append(e)
    wraps_h.sort(key=lambda e: GA.y[e.source()])
    wraps_v.sort(key=lambda e: GA.x[e.source()])
    
    for nr, e in enumerate(wraps_h + wraps_v):
        # GA.arrowType[e] = getattr(ogdf.EdgeArrow, "None")
        apply_wrap(GA, box, e, wraps[e], 50 + nr * 5)
        if wraps[e] in ["hv", "vh"]:
            GA.strokeColor[e] = ogdf.Color(0,255,0)
        elif wraps[e] == "h":
            GA.strokeColor[e] = ogdf.Color(255,0,0)
        elif wraps[e] == "v":
            GA.strokeColor[e] = ogdf.Color(0,0,255)
        
    return GA, wraps

def draw_bounding_box(G, GA, box):
    box_node1 = G.newNode()
    box_node2 = G.newNode()
    GA.x[box_node1] = box.p1().m_x - 25
    GA.y[box_node1] = box.p1().m_y - 25
    GA.x[box_node2] = box.p2().m_x + 25
    GA.y[box_node2] = box.p2().m_y + 25
    GA.width[box_node1] = 0
    GA.height[box_node1] = 0
    GA.width[box_node2] = 0
    GA.height[box_node2] = 0
    box_edge1 = G.newEdge(box_node1, box_node2)
    box_edge2 = G.newEdge(box_node1, box_node2)
    add_bends(GA.bends[box_edge1], (box.p1().m_x - 25, box.p2().m_y + 25))
    add_bends(GA.bends[box_edge2], (box.p2().m_x + 25, box.p1().m_y - 25))

env: OGDF_BUILD_DIR=~/OGDF-master/build-debug


In [2]:
width = 5
height = 5

G = ogdf.Graph()
ogdf.gridGraph(G, width, height, True, True)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
GA.destroyAttributes(ogdf.GraphAttributes.nodeId)

for n in G.nodes:
    GA.label[n] = str(n.index())
    GA.x[n] = (n.index() % width) * 50 
    GA.y[n] = (n.index() // height) * 50

box = GA.boundingBox()
GA, wraps = layout_edges(GA, box)
# draw_bounding_box(G, GA, box)
GA

In [3]:

G.delNode(G.nodes[5])
G.delNode(G.nodes[9])
G.delNode(G.nodes[15])
G.delNode(G.nodes[19])

#G.delEdge(G.searchEdge(G.nodes[5], G.nodes[6]))

# G.unsplit(G.nodes[15]) # replace deg 2 node by edge

G.newEdge(G.nodes[0],G.nodes[10])
G.newEdge(G.nodes[10],G.nodes[20])
G.newEdge(G.nodes[4],G.nodes[14])
G.newEdge(G.nodes[14],G.nodes[24])

layout_edges(GA, box)
GA

G.delNode(G.nodes[0])
G.delNode(G.nodes[1])
G.delNode(G.nodes[2])
G.delNode(G.nodes[3])
G.delNode(G.nodes[6])
G.delNode(G.nodes[10])
G.delNode(G.nodes[12])
G.delNode(G.nodes[14])
G.delNode(G.nodes[15])
G.newEdge(G.nodes[8],G.nodes[4])
G.newEdge(G.nodes[13],G.nodes[5])
G.newEdge(G.nodes[9],G.nodes[11])
G.newEdge(G.nodes[11],G.nodes[7])
G.unsplit(G.nodes[13]) # replace node of degree 2
layout_edges(GA, box)

GA

G.delNode(G.nodes[0])
G.delNode(G.nodes[1])
G.delNode(G.nodes[2])
G.delNode(G.nodes[4])
G.delNode(G.nodes[8])
G.delNode(G.nodes[12])
G.delNode(G.nodes[13])
G.delNode(G.nodes[11])
G.delNode(G.nodes[14])
G.delNode(G.nodes[15])


G.delEdge(G.searchEdge(G.nodes[5], G.nodes[6]))

# G.unsplit(G.nodes[15]) # replace deg 2 node by edge

G.newEdge(G.nodes[9],G.nodes[5])
G.newEdge(G.nodes[3],G.nodes[5])
G.newEdge(G.nodes[10],G.nodes[6])
G.newEdge(G.nodes[7],G.nodes[3])




layout_edges(GA, box)
GA

ogdf.GraphIO.drawSVG(GA, "test1.png")

In [4]:
#create combinatorial embedding by sorting edges around a node according to the graph

def adjList(nodes):
    
    adjList = {}
    
    for n in G.nodes:
        
        #adjList[GA.label[n]] = {}
        adjList[n] = {}
        
        for adj in n.adjEntries:
            
            e = adj.theEdge()
            wrap = find_wrap(GA, e)
            
            if n == e.source():
                
                if wrap == "h" or wrap == "hv":
                    position = 1
                    
                elif wrap == "v" or wrap == "vh":
                    position = 2
                    
                else:
                    if GA.x[n] == GA.x[adj.twinNode()]:
                        position = 2
                        
                    if GA.y[n] == GA.y[adj.twinNode()]:
                        position = 1
        
                #adjList[GA.label[n]][position] = GA.label[adj.twinNode()]  #uncomment to use node lab
                adjList[n][position] = adj
                        
            else:
                
                if wrap == "h" or wrap == "hv":
                    position = 3
                    
                elif wrap == "v" or wrap == "vh":
                    position = 0
                    
                else:
                    if GA.x[n] == GA.x[adj.twinNode()]:
                        position = 0
                        
                    if GA.y[n] == GA.y[adj.twinNode()]:
                        position = 3
                        
                #adjList[GA.label[n]][position] = GA.label[adj.twinNode()]  #uncomment to use node labels
                adjList[n][position] = adj
                
    
    return adjList


def sorted_adjList(adjList):
    
    sorted_adjList = {}
    
    for n in adjList:
        
        sortedList = []
        
        for i in sorted (adjList[n]) :
            
            sortedList.append(adjList[n][i])
        
        sorted_adjList[n] = sortedList
    
    return sorted_adjList
    

    
def sortAdjs(node, order):
    l = ogdf.List["ogdf::adjEntry"]()
    for adj in order:
        l.pushBack(adj)
    node.adjEntries.sort(l)


In [5]:
result = adjList(G.nodes)

sorted_adjList = sorted_adjList(result)

for n in sorted_adjList:
    
    node_adjEntries = sorted_adjList[n]
    sortAdjs(n, node_adjEntries)
    print(n.index(), [(e.theEdge().source().index(),e.theEdge().target().index()) for e in list(n.adjEntries)])

   

0 [(20, 0), (0, 1), (0, 10), (4, 0)]
1 [(21, 1), (1, 2), (1, 6), (0, 1)]
2 [(22, 2), (2, 3), (2, 7), (1, 2)]
3 [(23, 3), (3, 4), (3, 8), (2, 3)]
4 [(24, 4), (4, 0), (4, 14), (3, 4)]
6 [(1, 6), (6, 7), (6, 11)]
7 [(2, 7), (7, 8), (7, 12), (6, 7)]
8 [(3, 8), (8, 13), (7, 8)]
10 [(0, 10), (10, 11), (10, 20), (14, 10)]
11 [(6, 11), (11, 12), (11, 16), (10, 11)]
12 [(7, 12), (12, 13), (12, 17), (11, 12)]
13 [(8, 13), (13, 14), (13, 18), (12, 13)]
14 [(4, 14), (14, 10), (14, 24), (13, 14)]
16 [(11, 16), (16, 17), (16, 21)]
17 [(12, 17), (17, 18), (17, 22), (16, 17)]
18 [(13, 18), (18, 23), (17, 18)]
20 [(10, 20), (20, 21), (20, 0), (24, 20)]
21 [(16, 21), (21, 22), (21, 1), (20, 21)]
22 [(17, 22), (22, 23), (22, 2), (21, 22)]
23 [(18, 23), (23, 24), (23, 3), (22, 23)]
24 [(14, 24), (24, 20), (24, 4), (23, 24)]


In [6]:
G.genus()

1

In [7]:
#for computing the dual

import cppyy
dualG = ogdf.Graph()
# maps G.adjEntry -> dualG.node (face)
dualFace = ogdf.AdjEntryArray["ogdf::node"](G, cppyy.bind_object(cppyy.nullptr, "ogdf::NodeElement"))
# maps G.edge -> dualG.edge
dualEdge = ogdf.EdgeArray["ogdf::edge"](G, cppyy.bind_object(cppyy.nullptr, "ogdf::EdgeElement"))
# maps dualG.edge -> G.edge (== dualEdge^{-1})
primalEdge = ogdf.EdgeArray["ogdf::edge"](dualG, cppyy.bind_object(cppyy.nullptr, "ogdf::EdgeElement"))

for n in G.nodes:
    for adj in n.adjEntries:
        if dualFace[adj]:
            continue
        face = dualG.newNode()
        dualFace[adj] = face
        
        a = adj.faceCycleSucc()
        while a != adj:
            dualFace[a] = face
            a = a.faceCycleSucc()

for e in G.edges:
    de = dualG.newEdge(dualFace[e.adjSource()], dualFace[e.adjTarget()])
    dualEdge[e] = de
    primalEdge[de] = e

In [8]:
#for computing the spanning trees 

for e in G.edges:
    GA.strokeColor[e] = ogdf.Color(0,0,0)

# type of G edges: 0 (black) is in X, 1 (red) is edge of spanning cotree, 2 (green) is edge of primal ST
tree = ogdf.EdgeArray[int](G, 0)

found = ogdf.NodeArray[bool](dualG, False)
first_node = next(iter(dualG.nodes))
found[first_node] = True
todo = list(first_node.adjEntries)
count = 1
while len(todo) > 0:
    cur = todo.pop(-1)
    if found[cur.twinNode()]:
        continue

    tree[primalEdge[cur.theEdge()]] = 1
    GA.strokeColor[primalEdge[cur.theEdge()]] = ogdf.Color(255,0,0)
    found[cur.twinNode()] = True
    count += 1

    for adj in cur.twinNode().adjEntries:
        if not found[adj.twinNode()]:
            todo.append(adj)
print(count, dualG.numberOfNodes())

found = ogdf.NodeArray[bool](G, False)
first_node = next(iter(G.nodes))
found[first_node] = True
todo = list(first_node.adjEntries)
count = 1
while len(todo) > 0:
    cur = todo.pop(-1)
    if found[cur.twinNode()] or tree[cur] != 0:
        continue

    tree[cur.theEdge()] = 2
    GA.strokeColor[cur.theEdge()] = ogdf.Color(0,255,0)
    found[cur.twinNode()] = True
    count += 1

    for adj in cur.twinNode().adjEntries:
        if not found[adj.twinNode()] and tree[adj] == 0:
            todo.append(adj)

print(count, G.numberOfNodes())

unlabeled = [e for e in G.edges if tree[e] == 0]
print(unlabeled)
GA


19 19
21 21
[<cppyy.gbl.ogdf.EdgeElement object at 0x6856b00>, <cppyy.gbl.ogdf.EdgeElement object at 0x68574d8>]


ogdf.GraphIO.drawSVG(GA, "test1_treecotreedecomp.png")

In [9]:
import math
#initialize orientation (horizontal/vertical)
orientation = ogdf.EdgeArray[int](G,99)

#find a list of bends of an edge
def bendCoord(e):
    bends = [(b.m_x,b.m_y) for b in GA.bends[e]]
    return bends

#find angle relative to a node (180 'West', 90 'North', 0 'East', -90 'South')
def findAngle(node,x,y): #angle relative to the current node 
    refx = GA.x[node]
    refy = GA.y[node]
    rad = math.atan2(refy-y,x-refx)
    deg = math.degrees(rad)
    return deg

#find turn relative to refNode left -1/right 1 /straight 0
def findDirection(refNode,e1,e2): 
    
    if len(bendCoord(e1)) != 0:
                    
        if e1.target() == refNode:
                    
            bendpoint = bendCoord(e1)[-1]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
            
                    
            a1 = findAngle(refNode,bendpointx,bendpointy)
                        
        else:
            
            bendpoint = bendCoord(e1)[0]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
           
                    
            a1 = findAngle(refNode,bendpointx,bendpointy)
                        
    else:
        if e1.target() == refNode:
            nextNode = e1.source()
            a1 = findAngle(refNode,GA.x[nextNode],GA.y[nextNode])
        else:
            nextNode = e1.target()
            a1 = findAngle(refNode,GA.x[nextNode],GA.y[nextNode])
        
    if len(bendCoord(e2)) != 0:
               
        if e2.target() == refNode:
            bendpoint = bendCoord(e2)[-1]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
                    
            a2 = findAngle(refNode,bendpointx,bendpointy)
                        
        else:
            bendpoint = bendCoord(e2)[0]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
                    
            a2 = findAngle(refNode,bendpointx,bendpointy)
                        
    else:
        if e2.target() == refNode:
            nextNode = e2.source()
            a2 = findAngle(refNode,GA.x[nextNode],GA.y[nextNode])
           
        else:
            nextNode = e2.target()
            a2 = findAngle(refNode,GA.x[nextNode],GA.y[nextNode])
            
    
    if a1 == 180: 
        if a2 == 90:
            direction = -1
        elif a2 == 0:
            direction = 0
        elif a2 == -90:
            direction = 1
            
    elif a1 == -90:
        if a2 == 90:
            direction = 0
        elif a2 == 0:
            direction = 1
        elif a2 == 180:
            direction = -1
            
    elif a1 == 90:
        if a2 == -90:
            direction = 0
        elif a2 == 0:
            direction = -1
        elif a2 == 180:
            direction = 1
            
    elif a1 == 0:
        if a2 == -90:
            direction = -1
        elif a2 == 90:
            direction = 1
        elif a2 == 180:
            direction = 0
    print(a1,a2)
    return direction

#vertical or horizontal (0/1)
def defineOrientation(refEdge,refNode,refOrientation): 
    
    
    if len(bendCoord(refEdge)) != 0:
        if refEdge.target() == refNode:          
            bendpoint = bendCoord(refEdge)[-1]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
                    
            refAngle = findAngle(refNode,bendpointx,bendpointy)
                        
        else:
            bendpoint = bendCoord(refEdge)[0]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
                    
            refAngle = findAngle(refNode,bendpointx,bendpointy)
                        
    else:
        if refEdge.target() == refNode:        
            refAngle = findAngle(refNode,GA.x[refEdge.source()],GA.y[refEdge.source()])
                        
        else:         
            refAngle = findAngle(refNode,GA.x[refEdge.target()],GA.y[refEdge.target()])
        
    
    
    for adj in refNode.adjEntries:
        e= adj.theEdge()
        nextNode = adj.twinNode()
        #print(refNode.index(),nextNode.index())
        if e!= refEdge:
            if orientation[e] == 0 or orientation [e] == 1: #to distinguish vertical 1 and horizontal 0
                #print("this edge is done")
                continue
                
            else:
                if len(bendCoord(e)) != 0:
                    #print("has bend")
                    if e.target() == refNode:
                    
                        bendpoint = bendCoord(e)[-1]
                        bendpointx = bendpoint[0]
                        bendpointy = bendpoint[1]
                    
                        angle = findAngle(refNode,bendpointx,bendpointy)
                        
                    else:
                        bendpoint = bendCoord(e)[0]
                        bendpointx = bendpoint[0]
                        bendpointy = bendpoint[1]
                    
                        angle = findAngle(refNode,bendpointx,bendpointy)
                        
                else:
                    angle = findAngle(refNode,GA.x[nextNode],GA.y[nextNode])
                    
            
            if abs(angle)%180 == abs(refAngle)%180:
                orientation[e] = refOrientation
                    
            else: 
                orientation[e] = 1-refOrientation
                    
            
                    
            defineOrientation(e,nextNode,orientation[e])
    
    return
   
        
#find the edges in a generator       
def bfs_findGenerator(target, queue, done = [], path = {}):
    #print("target", target.index())
    while queue:
        
        try:
            currEdge = queue[0][0].theEdge()
            
            (currAdj, currNode) = queue.pop(0)
            currEdge = currAdj.theEdge()
            path[currNode] = []
            done.append(currEdge)
        except:
            (currEdge, currNode) = queue.pop(0)
            path[currNode] = []
            done.append(currEdge)

        #print(currNode.index())
        for adj in currNode.adjEntries:
            e = adj.theEdge()
            if e not in done and tree[e] == 2:
                path[currNode].append(adj)
                #print("curr node:", currNode.index())
                #print("adj:", adj.theEdge().source().index(),adj.theEdge().target().index())
                if adj.twinNode() == target:
                    #print("found target", adj.twinNode().index())
                    return path
            
                queue.append((e, adj.twinNode()))
                
    print("no path found")
        
def checkCoeff(loop, variableList):
    if len(loop) > 0:
        sum = 0
        for e in loop:
            sum += variableList[e]
            
        if sum < 0:
            rhs = -1
        else:
            rhs = 1
            
    else:
        rhs = 0
        
    return rhs

In [15]:
#the black edges that must be included in each generator
starter_edges = [e for e in G.edges if tree[e] == 0]


start_edge0 = starter_edges[0]
target0 = start_edge0.target()
source0 =  start_edge0.source()
#print(source0.index(), target0.index())
queue = [(start_edge0, target0)]
path0 = bfs_findGenerator(source0, queue, done = [], path = {})
betaLoop = []
betaLoop.append(start_edge0)
prev_node = None

for e in reversed(path0):
    if prev_node == None:
        #print(e.index(), path0[e][-1].theEdge().source().index(),path0[e][-1].theEdge().target().index())
        betaLoop.append(path0[e][-1].theEdge())
        prev_node = e
    else:
        for adj in path0[e]:
            if adj.twinNode() == prev_node:
                betaLoop.append(adj.theEdge())
                print(adj.twinNode().index())
                prev_node = e
                break

print("beta loop")
for i, e in enumerate(betaLoop):
    print(betaLoop[i].source().index(),betaLoop[i].target().index())

    

start_edge1 = starter_edges[1]
target1 = start_edge1.target()
source1 =  start_edge1.source()
#print(source1.index(), target1.index())
queue = [(start_edge1, target1)]
path1 = bfs_findGenerator(source1, queue, done = [], path = {})
alphaLoop = []
alphaLoop.append(start_edge1)
prev_node = None

for e in reversed(path1):
    if prev_node == None:
        alphaLoop.append(path1[e][-1].theEdge())
        prev_node = e
    else:
        for adj in path1[e]:
            if adj.twinNode() == prev_node:
                alphaLoop.append(adj.theEdge())
                prev_node = e
                break
print("alpha loop")
for i, e in enumerate(alphaLoop):
    print(alphaLoop[i].source().index(),alphaLoop[i].target().index())

4
3
2
beta loop
0 1
4 0
3 4
2 3
1 2
alpha loop
20 0
10 20
10 11
11 12
7 12
7 8
3 8
3 4
4 0


for n in path1:
    print("node", n.index())
    for adj in path1[n]:
        e = adj.theEdge()
        print(e.source().index(),e.target().index())

In [21]:
defineOrientation(start_edge0,start_edge0.source(),0)
for e in G.edges:
    print(e.source().index(), e.target().index(), orientation[e],find_wrap(GA,e))

0 1 0 
1 2 0 
2 3 0 
3 4 0 
4 0 0 h
1 6 1 
6 7 0 
2 7 1 
7 8 0 
3 8 1 
10 11 0 
6 11 1 
11 12 0 
7 12 1 
12 13 0 
8 13 1 
13 14 0 
14 10 0 h
11 16 1 
16 17 0 
12 17 1 
17 18 0 
13 18 1 
20 21 0 
16 21 1 
21 22 0 
17 22 1 
22 23 0 
18 23 1 
23 24 0 
24 20 0 h
20 0 1 v
21 1 1 v
22 2 1 v
23 3 1 v
24 4 1 v
0 10 1 
10 20 1 
4 14 1 
14 24 1 


#the black edges that must be included in each generator
starter_edges = [e for e in G.edges if tree[e] == 0]


start_edge = starter_edges[0]
betaLoop = [start_edge]
bool, betaLoop = findGenerators(start_edge, start_edge.source(),start_edge.target(), betaLoop)
print("beta loop")
for i, e in enumerate(betaLoop):
    print(betaLoop[i].source().index(),betaLoop[i].target().index())

print("alpha loop")
start_edge = starter_edges[1]
alphaLoop = [start_edge]
bool, alphaLoop = findGenerators(start_edge, start_edge.source(),start_edge.target(),alphaLoop)
for i, e in enumerate(alphaLoop):
    print(alphaLoop[i].source().index(),alphaLoop[i].target().index())

for i, e in enumerate(alphaLoop):
    if i < len(alphaLoop)-1:
        print(e.source().index(),e.target().index(), findDirection(e.commonNode(alphaLoop[i+1]),alphaLoop[i],alphaLoop[i+1]))
    else:
        print(alphaLoop[i].source().index(),alphaLoop[i].target().index(), findDirection(e.commonNode(alphaLoop[0]),alphaLoop[i],alphaLoop[0]))
        
for i, e in enumerate(betaLoop):
    if i < len(betaLoop)-1:
        print(e.source().index(),e.target().index(), findDirection(e.commonNode(betaLoop[i+1]),betaLoop[i],betaLoop[i+1]))
    else:
        print(betaLoop[i].source().index(),betaLoop[i].target().index(), findDirection(e.commonNode(betaLoop[0]),betaLoop[i],betaLoop[0]))
        


In [22]:
from pulp import *
import pandas as pd
import numpy as np

#give an orientation to each edge
alphaLoopVertical = []
alphaLoopHorizontal = []
signs1 = {}
flags = {}
prev_turn = 999
for i, e in enumerate(alphaLoop):
    
    if i < len(alphaLoop)-1:
        ei = alphaLoop[i]
        ei1 = alphaLoop[i+1]           
    else:
        ei = alphaLoop[i]
        ei1 = alphaLoop[0]
            
    turn = findDirection(e.commonNode(ei1),e,ei1)
    
    if prev_turn == 999:
        flag = 1 
        sign = 1
        flags[1] = sign
    elif turn !=0 and turn == prev_turn:
        flag = -1*flag
        sign = -1*flags[flag]
        flags[flag] = sign
    elif turn !=0 and turn != prev_turn:
        flag = -1*flag
        if flag in flags:
            sign = flags[flag]
        else:
            flags[flag] = 1
            sign =1
    else:
        sign = flags[flag]
            
        
    if orientation[e] == 1:
        alphaLoopHorizontal.append(e)
    else:
        alphaLoopVertical.append(e)
    
    signs1[ei1] = sign
    
    prev_turn = turn
    prev_sign = sign
    
    print(ei1.source().index(),ei1.target().index(), signs1[ei1])


-90.0 90.0
10 20 1
-90.0 0.0
10 11 1
180.0 0.0
11 12 1
180.0 90.0
7 12 1
-90.0 0.0
7 8 1
180.0 90.0
3 8 1
-90.0 0.0
3 4 1
180.0 0.0
4 0 1
180.0 90.0
20 0 1


In [23]:
betaLoopVertical = []
betaLoopHorizontal = []
signs2 = {}
flags = {}
prev_turn = 999
for i, e in enumerate(betaLoop):
    
    if i < len(betaLoop)-1:
        ei = betaLoop[i]
        ei1 = betaLoop[i+1]           
    else:
        ei = betaLoop[i]
        ei1 = betaLoop[0]
            
    turn = findDirection(e.commonNode(ei1),e,ei1)
    
    if prev_turn == 999:
        flag = 1 
        sign = 1
        flags[1] = sign
    elif turn !=0 and turn == prev_turn:
        flag = -1*flag
        sign = -1*flags[flag]
        flags[flag] = sign
    elif turn !=0 and turn != prev_turn:
        flag = -1*flag
        if flag in flags:
            sign = flags[flag]
        else:
            flags[flag] = 1
            sign =1
    else:
        sign = flags[flag]
            
        
    if orientation[e] == 1:
        betaLoopHorizontal.append(e)
    else:
        betaLoopVertical.append(e)
    
    signs2[ei1] = sign
    
    prev_turn = turn
    prev_sign = sign
    
    print(ei1.source().index(),ei1.target().index(), signs2[ei1])



0.0 180.0
4 0 1
0.0 180.0
3 4 1
0.0 180.0
2 3 1
0.0 180.0
1 2 1
0.0 180.0
0 1 1


In [24]:
for e in betaLoopHorizontal:
    print(e.source().index(),e.target().index())

In [25]:
##Setting variable names

alphaVerticalDecisionVariables = {}
betaVerticalDecisionVariables = {}
alphaHorizontalDecisionVariables = {}
betaHorizontalDecisionVariables = {}

for i, e in enumerate(alphaLoopVertical):
    var = LpVariable('eav' + str(i) ,lowBound = 0.1) 
    alphaVerticalDecisionVariables[e] = var
    
    if e in betaLoopVertical:
        betaVerticalDecisionVariables[e] = var
    
for i, e in enumerate(betaLoopVertical):
    var = LpVariable('ebv' + str(i) ,lowBound = 0.1) 
    if e not in alphaLoopVertical:
        betaVerticalDecisionVariables[e] = var

for i, e in enumerate(alphaLoopHorizontal):
    var = LpVariable('eah' + str(i) ,lowBound = 0.1) 
    alphaHorizontalDecisionVariables[e] = var
    
    if e in betaLoopHorizontal:
        betaHorizontalDecisionVariables[e] = var
    
for i, e in enumerate(betaLoopHorizontal):
    var = LpVariable('ebh' + str(i) ,lowBound = 0.1) 
    #print(e.source().index(),e.target().index())
    if e not in alphaLoopHorizontal:
        #print(e.source().index(),e.target().index())
        betaHorizontalDecisionVariables[e] = var
        

In [26]:
#check if variable names are consistent

print("betaLoopHorz")
beta_horz_rhs = 0
if len(betaHorizontalDecisionVariables) != 0:
    #beta_horz_rhs = 0
    for e in betaHorizontalDecisionVariables:
        if find_wrap(GA,e) != "":
            print(find_wrap(GA,e))
            beta_horz_rhs += 1
        print(e.source().index(),e.target().index(), betaHorizontalDecisionVariables[e])
print(beta_horz_rhs)

print("alphaLoopHorz")
alpha_horz_rhs = 0
if len(alphaHorizontalDecisionVariables) != 0:
    #alpha_horz_rhs = 1
    for e in alphaHorizontalDecisionVariables:
        if find_wrap(GA,e) != "":
            alpha_horz_rhs += 1
        print(e.source().index(),e.target().index(), alphaHorizontalDecisionVariables[e])
beta_ver_rhs = 0
print(alpha_horz_rhs)

print("betaLoopVer")
beta_ver_rhs = 0
if len(betaVerticalDecisionVariables) != 0:
    #beta_ver_rhs = 1
    for e in betaVerticalDecisionVariables:
        if find_wrap(GA,e) != "":
            beta_ver_rhs += 1
        print(e.source().index(),e.target().index(),betaVerticalDecisionVariables[e])
else:
    beta_ver_rhs = 0
print(beta_ver_rhs)
        
print("alphaLoopVer")
alpha_ver_rhs = 0
if len(alphaVerticalDecisionVariables) != 0:
    for e in alphaVerticalDecisionVariables:
        if find_wrap(GA,e) != "":
            alpha_ver_rhs += 1
        print(e.source().index(),e.target().index(),alphaVerticalDecisionVariables[e])
print(alpha_ver_rhs)

betaLoopHorz
0
alphaLoopHorz
20 0 eah0
10 20 eah1
7 12 eah2
3 8 eah3
1
betaLoopVer
3 4 eav3
4 0 eav4
0 1 ebv0
2 3 ebv3
1 2 ebv4
1
alphaLoopVer
10 11 eav0
11 12 eav1
7 8 eav2
3 4 eav3
4 0 eav4
1


In [27]:
setOfVariables = set(list(betaHorizontalDecisionVariables.values()) + list(alphaHorizontalDecisionVariables.values()) +list(betaVerticalDecisionVariables.values())+list(alphaVerticalDecisionVariables.values()))

model= LpProblem("Network Flow Problem", LpMinimize)

# The objective function
model += lpSum([e for e in setOfVariables]), "Sum of flow across all edges"

# Constraints, if there are more edges going the 'opposite' direction we multiply it by -1 so that a solution exists
if checkCoeff(betaLoopHorizontal, signs2) == 1:  
    model += lpSum([signs2[e]*betaHorizontalDecisionVariables[e] for e in betaHorizontalDecisionVariables.keys() ]) == beta_horz_rhs, "Sum of flow across all horizontal edges in beta loop"    
elif checkCoeff(betaLoopHorizontal, signs2) == -1:  
    model += lpSum([signs2[e]*betaHorizontalDecisionVariables[e] for e in betaHorizontalDecisionVariables.keys() ]) == -1*beta_horz_rhs, "Sum of flow across all horizontal edges in beta loop"
    
if checkCoeff(alphaLoopVertical, signs1) == 1:  
    model += lpSum([signs1[e]*alphaVerticalDecisionVariables[e] for e in alphaVerticalDecisionVariables.keys() ]) == alpha_ver_rhs, "Sum of flow across all vertical edges in alpha loop"
elif checkCoeff(alphaLoopVertical, signs1) == -1:  
    model += lpSum([signs1[e]*alphaVerticalDecisionVariables[e] for e in alphaVerticalDecisionVariables.keys() ]) == -1*alpha_ver_rhs, "Sum of flow across all vertical edges in alpha loop"

if checkCoeff(betaLoopVertical, signs2) == 1:    
    model += lpSum([signs2[e]*betaVerticalDecisionVariables[e] for e in betaVerticalDecisionVariables.keys() ]) == beta_ver_rhs, "Sum of flow across all vertical edges in beta loop"
else:
    model += lpSum([signs2[e]*betaVerticalDecisionVariables[e] for e in betaVerticalDecisionVariables.keys() ]) == -1*beta_ver_rhs, "Sum of flow across all vertical edges in beta loop"

if checkCoeff(alphaLoopHorizontal, signs1) == 1: 
    model += lpSum([signs1[e]*alphaHorizontalDecisionVariables[e] for e in alphaHorizontalDecisionVariables.keys() ]) == alpha_horz_rhs, "Sum of flow across all horizontal edges in alpha loop"
else:
    model += lpSum([signs1[e]*alphaHorizontalDecisionVariables[e] for e in alphaHorizontalDecisionVariables.keys() ]) == -1*alpha_horz_rhs, "Sum of flow across all horizontal edges"
    

# the following needs to be fixed
restDecisionVariables = {}
edge_set_list = []
for i,n in enumerate(G.nodes):
    
    for k,adj in enumerate(n.adjEntries):
        lhs1 = []
        rhs1 = []
        lhs0 = []
        rhs0 = []
        edge_set = set()
        e = adj.theEdge()
        edge_set.add(e)
        list = lhs1
        list.append(e)
        orig_adj = adj
        #print("current edge",e.source().index(),e.target().index())
        
        while True:
            
            next_adj = adj.faceCycleSucc()
            next_e = next_adj.theEdge()
            edge_set.add(next_e)
            #print("next edge", next_e.source().index(),next_e.target().index(),orientation[next_e])
            
            if next_adj == orig_adj:
                break
                
            if orientation[next_e] == orientation[e]:
                list.append(next_e)
                
            else:
                if lhs0 == []:
                    lhs0.append(next_e)
                    list = lhs0
                elif rhs1 == []:
                    rhs1.append(next_e)
                    list = rhs1
                elif rhs0 == []:
                    rhs0.append(next_e)
                    list = rhs0
                else:
                    lhs1.append(next_e)
                    list = lhs1


            e = next_e
            adj = next_adj
    
        if edge_set in edge_set_list:
            continue
        else:
            edge_set_list.append(edge_set)       
        
        #print("rhs0")
        #[print(e.source().index(),e.target().index()) for e in rhs0]
        #print("lhs0")
        #[print(e.source().index(),e.target().index()) for e in lhs0]
        #print("rhs1")
        #[print(e.source().index(),e.target().index()) for e in rhs1]
        #print("lhs1")
        #[print(e.source().index(),e.target().index()) for e in lhs1]
        for j, e in enumerate(lhs1):
            if e in restDecisionVariables:
                pass      
            elif e in alphaHorizontalDecisionVariables:
                varName = alphaHorizontalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            elif e in alphaVerticalDecisionVariables:
                varName = alphaVerticalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            elif e in betaHorizontalDecisionVariables:
                varName = betaHorizontalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            elif e in betaVerticalDecisionVariables:
                varName = betaVerticalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            else:
                varName = LpVariable('e11' + str(i) + str(k) + str(j) ,lowBound = 0.1) 
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            
        for j,e in enumerate(rhs1):
            if e in restDecisionVariables:
                pass      
            elif e in alphaHorizontalDecisionVariables:
                varName = alphaHorizontalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            elif e in alphaVerticalDecisionVariables:
                varName = alphaVerticalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            elif e in betaHorizontalDecisionVariables:
                varName = betaHorizontalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            elif e in betaVerticalDecisionVariables:
                varName = betaVerticalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            else:
                varName = LpVariable('e12' + str(i) + str(k) + str(j) ,lowBound = 0.1)
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            
        for j, e in enumerate(lhs0):
            if e in restDecisionVariables:
                pass      
            elif e in alphaHorizontalDecisionVariables:
                varName = alphaHorizontalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            elif e in alphaVerticalDecisionVariables:
                varName = alphaVerticalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            elif e in betaHorizontalDecisionVariables:
                varName = betaHorizontalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            elif e in betaVerticalDecisionVariables:
                varName = betaVerticalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            else:
                varName = LpVariable('e21' + str(i) + str(k) + str(j) ,lowBound = 0.1) 
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            
        for j,e in enumerate(rhs0):
            if e in restDecisionVariables:
                pass      
            elif e in alphaHorizontalDecisionVariables:
                varName = alphaHorizontalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            elif e in alphaVerticalDecisionVariables:
                varName = alphaVerticalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            elif e in betaHorizontalDecisionVariables:
                varName = betaHorizontalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            elif e in betaVerticalDecisionVariables:
                varName = betaVerticalDecisionVariables[e]
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            else:
                varName = LpVariable('e22' + str(i) + str(k) + str(j) ,lowBound = 0.1)
                restDecisionVariables[e] = varName
                print(e.source().index(),e.target().index(),varName)
            
        print("rhs0")
        [print(e.source().index(),e.target().index()) for e in rhs0]
        print("lhs0")
        [print(e.source().index(),e.target().index()) for e in lhs0]
        print("rhs1")
        [print(e.source().index(),e.target().index()) for e in rhs1]
        print("lhs1")
        [print(e.source().index(),e.target().index()) for e in lhs1] 
        if set(lhs1) == set(rhs1):
            print("pass")
            pass
        else:
            model += lpSum([restDecisionVariables[e] for e in lhs1 ]) == lpSum([restDecisionVariables[e] for e in rhs1 ]), "equal flow on left/right face {} {}".format(i,k)
        if set(lhs0) == set(lhs1):
            print("pass")
            pass
        else:
            model += lpSum([restDecisionVariables[e] for e in lhs0 ]) == lpSum([restDecisionVariables[e] for e in rhs0 ]), "equal flow on top/bottom face {} {}".format(i,k)
    

20 0 eah0
21 1 e12000
20 21 e21000
0 1 ebv0
rhs0
0 1
lhs0
20 21
rhs1
21 1
lhs1
20 0
10 11 eav0
1 6 e21010
6 11 e21011
0 10 e22010
rhs0
0 10
lhs0
1 6
6 11
rhs1
10 11
lhs1
0 1
4 14 e12020
14 10 e21020
4 0 eav4
rhs0
4 0
lhs0
14 10
rhs1
4 14
lhs1
0 10
24 20 e12030
24 4 e21030
rhs0
20 0
lhs0
24 4
rhs1
24 20
lhs1
4 0
22 2 e12100
21 22 e21100
1 2 ebv4
rhs0
1 2
lhs0
21 22
rhs1
22 2
lhs1
21 1
6 7 e12110
2 7 e21110
rhs0
1 6
lhs0
2 7
rhs1
6 7
lhs1
1 2
23 3 e12200
22 23 e21200
2 3 ebv3
rhs0
2 3
lhs0
22 23
rhs1
23 3
lhs1
22 2
7 8 eav2
3 8 eah3
rhs0
2 7
lhs0
3 8
rhs1
7 8
lhs1
2 3
23 24 e21300
3 4 eav3
rhs0
3 4
lhs0
23 24
rhs1
24 4
lhs1
23 3
13 14 e12310
8 13 e22310
rhs0
8 13
3 8
lhs0
4 14
rhs1
13 14
lhs1
3 4
11 12 eav1
7 12 eah2
rhs0
6 11
lhs0
7 12
rhs1
11 12
lhs1
6 7
12 13 e12610
rhs0
7 12
lhs0
8 13
rhs1
12 13
lhs1
7 8
11 16 e21810
16 21 e21811
10 20 eah1
rhs0
10 20
lhs0
11 16
16 21
rhs1
20 21
lhs1
10 11
14 24 e12820
rhs0
14 10
lhs0
24 20
rhs1
14 24
lhs1
10 20
16 17 e12910
12 17 e21910
rhs0
11 16
l

/home/loke/.local/lib/python3.9/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [28]:
model.solve()
print("Status:", LpStatus[model.status])

# Each of the variables is printed with its resolved optimum value
for v in model.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("The sum of flow = ", value(model.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/loke/.local/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/eef555338b684dbcbe7853625292f1f4-pulp.mps branch printingOptions all solution /tmp/eef555338b684dbcbe7853625292f1f4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 46 COLUMNS
At line 153 RHS
At line 195 BOUNDS
At line 236 ENDATA
Problem MODEL has 41 rows, 40 columns and 94 elements
Coin0008I MODEL read with 0 errors
Presolve 2 (-39) rows, 7 (-33) columns and 7 (-87) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 9.4100543e-07 ( 9.4100543e-05%) - largest zero change 0
0  Obj 1.3000006 Primal inf 0.999998 (2)
2  Obj 2.3000015
Optimal - objective value 2.3
After Postsolve, objective 2.3, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 2.3 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time 

# Draw the graph based on LP solution

In [29]:
def findAngle2(node1,node2, refEdge): #angle relative to the current node 
    #print("current node coords",node1.index(), node2.index(), GA.x[node1], GA.y[node1])
    for adj in node1.adjEntries:
        if adj.theEdge() == refEdge:
            #print(refEdge.source().index(), refEdge.target().index())
            if len(bendCoord(refEdge)) != 0:
                if refEdge.target() == node1:          
                    bendpoint = bendCoord(refEdge)[-1]
                    bendpointx = bendpoint[0]
                    bendpointy = bendpoint[1]
                    
                    refAngle = findAngle(node1,bendpointx,bendpointy)
                        
                else:
                    bendpoint = bendCoord(refEdge)[0]
                    bendpointx = bendpoint[0]
                    bendpointy = bendpoint[1]
                    
                    refAngle = findAngle(node1,bendpointx,bendpointy)
                
                #print("has bend", bendpointx, bendpointy)
                        
            else:
                refAngle = findAngle(node1,GA.x[node2],GA.y[node2])
                #print("no bend", GA.x[node2], GA.y[node2])
                
            return refAngle

In [30]:
coordinates = {}

def findCoords(coordinates):
    
    for n in G.nodes: 
        
        if coordinates == {}:
            x = 0
            y = 0
            coordinates[n] = (x,y)
            print("start index",x,y)
            
        if n not in coordinates:
            continue
            
        for adj in n.adjEntries:
            
            adjNode = adj.twinNode()
            e = adj.theEdge()
            if adjNode in coordinates:
                continue
                
            else:
                
                varName = restDecisionVariables[e]
                varValue = varName.varValue
            
                angle = findAngle2(n,adjNode,e)
                
                if angle == 90:
                    x = coordinates[n][0]
                    y = (coordinates[n][1] - varValue) % 1
                    print((coordinates[n][1] - varValue))
                elif angle == 0:
                    x = (coordinates[n][0] + varValue) % 1
                    y = coordinates[n][1]
                    #print(angle,varValue,x,y)
                elif angle == 180:
                    x = (coordinates[n][0] - varValue) % 1
                    y = coordinates[n][1]
                    #print(angle,varValue,x,y)
                elif angle == -90:
                    x = coordinates[n][0]
                    y = (coordinates[n][1] + varValue) % 1
                    #print(angle,varValue,x,y)
            
                coordinates[adjNode] = (x,y)
                
                #print(angle,varValue,x,y)

In [31]:
findCoords(coordinates)

#for e in restDecisionVariables:
 #   print(restDecisionVariables[e],e.source().index(),e.target().index())
    
#for n in coordinates:
 #   print(n.index(),coordinates[n][0], coordinates[n][1])

start index 0 0
-0.1
-0.1
-0.1
-0.1
-0.1


for e in restDecisionVariables:
    print(restDecisionVariables[e],e.source().index(),e.target().index())

In [32]:
for n in G.nodes:
    print(n.index())
    GA.x[n] = coordinates[n][0]* 1000
    GA.y[n] = coordinates[n][1]* 1000
    print(n.index(),GA.x[n],GA.y[n])

0
0 0.0 0.0
1
1 100.0 0.0
2
2 200.0 0.0
3
3 300.00000000000006 0.0
4
4 400.0 0.0
6
6 100.0 600.0
7
7 200.0 600.0
8
8 300.00000000000006 600.0
10
10 0.0 700.0
11
11 100.0 700.0
12
12 200.0 700.0
13
13 300.00000000000006 700.0
14
14 400.0 700.0
16
16 100.0 799.9999999999999
17
17 200.0 799.9999999999999
18
18 300.00000000000006 799.9999999999999
20
20 0.0 900.0
21
21 100.0 900.0
22
22 200.0 900.0
23
23 300.00000000000006 900.0
24
24 400.0 900.0


In [33]:
for e in G.edges:
    GA.bends[e].clear()

In [34]:

box = GA.boundingBox()
GA, wraps = layout_edges(GA, box)
# draw_bounding_box(G, GA, box)
GA

ogdf.GraphIO.drawSVG(GA, "test1_drawing.png")

restDecisionVariables = {}

for i,n in enumerate(G.nodes):
    prev_e = None
    print("current node", n.index())
    for adj in n.adjEntries:
        lhs1 = []
        rhs1 = []
        lhs0 = []
        rhs0 = []
        e = adj.theEdge()
        list = lhs1
        list.append(e)
        orig_adj = adj
        print("current edge",e.source().index(),e.target().index())
        
        while True:
            
            next_adj = adj.faceCycleSucc()
            next_e = next_adj.theEdge()
            print("next edge", next_e.source().index(),next_e.target().index(),orientation[next_e])
            
            if next_adj == orig_adj:
                break
                
            if orientation[next_e] == orientation[e]:
                list.append(next_e)
                
            else:
                if lhs0 == []:
                    lhs0.append(next_e)
                    list = lhs0
                elif rhs1 == []:
                    rhs1.append(next_e)
                    list = rhs1
                elif rhs0 == []:
                    rhs0.append(next_e)
                    list = rhs0
                else:
                    lhs1.append(next_e)
                    list = lhs1


            e = next_e
            adj = next_adj
        print("rhs0")
        [print(e.source().index(),e.target().index()) for e in rhs0]
        print("lhs0")
        [print(e.source().index(),e.target().index()) for e in lhs0]
        print("rhs1")
        [print(e.source().index(),e.target().index()) for e in rhs1]
        print("lhs1")
        [print(e.source().index(),e.target().index()) for e in lhs1]